In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [4]:
import sys
import os

# Step 1: Add the parent directory to sys.path
parent_dir = os.path.abspath("..")  # Get the absolute path of the parent directory
if parent_dir not in sys.path:
    sys.path.append(parent_dir)  # Add to sys.path if not already added


In [5]:
from models.yolo import Model
from utils.general import intersect_dicts

## 2. Export

PyTorch models are not directly compatible with the [QuantizeML quantization
tool](../../api_reference/quantizeml_apis.html)_, it is therefore necessary
to use an intermediate format. Like many other machine learning frameworks,
PyTorch has tools to export modules in the [ONNX](https://onnx.ai)_ format.

Therefore, the model is exported by the following code:




In [6]:
architecture = 'yolov5n'
device = 'cpu'

In [7]:
weights = '/Users/benediktwitteler/Code/Python/NeuromorphicHackathon/Object-tracking/finetuning/finetunedModels/yolov5n_mse_finetune4layers_model_last_epoch50.pth'

# Use specified newer YOLO model
model = Model(os.path.join('..', 'models', architecture+'.yaml')).to(device)
ckpt = torch.load(weights, map_location='cpu')  # load checkpoint to CPU to avoid CUDA memory leak
model.load_state_dict(ckpt)  # load
model.eval()


                 from  n    params  module                                  arguments                     
  0                -1  1      1760  models.common.Conv                      [3, 16, 6, 2, 2]              
  1                -1  1      4672  models.common.Conv                      [16, 32, 3, 2]                
  2                -1  1      4800  models.common.C3                        [32, 32, 1]                   
  3                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  4                -1  2     29184  models.common.C3                        [64, 64, 2]                   
  5                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  6                -1  3    156928  models.common.C3                        [128, 128, 3]                 
  7                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  8                -1  1    296448  

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
     

In [8]:
dummy_input = torch.randn(1,3,256,256)
torch.onnx.export(model,
                  dummy_input,
                  f="initial_model.onnx",
                  input_names=["inputs"],
                  output_names=["outputs"],
                  dynamic_axes={'inputs': {0: 'batch_size'}, 'outputs': {0: 'batch_size'}})

/Users/benediktwitteler/Code/Python/NeuromorphicHackathon/Object-tracking/models/yolo.py:269: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
/Users/benediktwitteler/Code/Python/NeuromorphicHackathon/Object-tracking/models/yolo.py:168: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
/Users/benediktwitteler/Code/Python/NeuromorphicHackathon/Object-tracking/models/yolo.py:172: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will

.. Note::
 Find more information about how to export PyTorch models in ONNX at
 [](https://pytorch.org/docs/stable/onnx.html).




## 3. Quantize

An Akida accelerator processes integer activations and weights. Therefore, the floating
point model must be quantized in preparation to run on an Akida accelerator.

The [QuantizeML quantize()](../../api_reference/quantizeml_apis.html#quantizeml.models.quantize)_
function recognizes [ModelProto](https://onnx.ai/onnx/api/classes.html#modelproto)_ objects
and can quantize them for Akida. The result is another ``ModelProto``, compatible with the
[CNN2SNN Toolkit](../../user_guide/cnn2snn.html)_.

.. Warning::
 ONNX and PyTorch offer their own quantization methods. You should not use those when preparing
 your model for Akida. Only the [QuantizeML quantize()](../../api_reference/quantizeml_apis.html#quantizeml.models.quantize)_ function
 can be used to generate a quantized model ready for conversion to Akida.

.. Note::
 For this simple model, using random samples for calibration is sufficient, as
 shown in the following steps.




In [1]:
import onnx


In [ ]:
from quantizeml.models import quantize


In [ ]:
# Read the exported ONNX model
model_onnx = onnx.load_model("initial_model.onnx")

# Extract a batch of train samples for calibration
calib_samples = dummy_input[0].numpy()

# Quantize
model_quantized = quantize(model_onnx, samples=calib_samples)
print(onnx.helper.printable_graph(model_quantized.graph))

## 4. Convert




### 4.1 Convert to Akida model

The quantized model can now be converted to the native Akida format.
The [convert()](../../api_reference/cnn2snn_apis.html#cnn2snn.convert)_
function returns a model in Akida format ready for inference.




In [ ]:
from cnn2snn import convert

model_akida = convert(model_quantized)
model_akida.summary()

### 4.2. Check performance

Native PyTorch data must be presented in a different format to perform
the evaluation in Akida models. Specifically:

1. images must be numpy-raw, with an 8-bit unsigned integer data type and
2. the channel dimension must be in the last dimension.




In [ ]:
# Read raw data and convert it into numpy
x_test = testloader.dataset.data.numpy()
y_test = testloader.dataset.targets.numpy()

# Add a channel dimension to the image sets as Akida expects 4-D inputs corresponding to
# (num_samples, width, height, channels). Note: MNIST is a grayscale dataset and is unusual
# in this respect - most image data already includes a channel dimension, and this step will
# not be necessary.
x_test = x_test[..., None]
y_test = y_test[..., None]

accuracy = model_akida.evaluate(x_test, y_test)
print('Test accuracy after conversion:', accuracy)

# For non-regression purposes
assert accuracy > 0.96

### 4.3 Show predictions for a single image

Display one of the test images, such as the first image in the aforementioned
dataset, to visualize the output of the model.




In [ ]:
# Test a single example
sample_image = 0
image = x_test[sample_image]
outputs = model_akida.predict(image.reshape(1, 28, 28, 1))

plt.imshow(x_test[sample_image].reshape((28, 28)), cmap="Greys")
print('Input Label:', y_test[sample_image].item())
print('Prediction Label:', outputs.squeeze().argmax())